In [ ]:
from glob import glob
import os
import json
from PIL import Image
import cv2, imageio
import numpy as np
from tqdm.auto import tqdm
annos = sorted(glob('fixed_data_08_swoon/*.json'))
imgs = sorted(glob('fixed_data_08_swoon/*.jpg'))
pairs = list(zip(annos, imgs))
assert len(annos) == len(imgs)
pairs[-1]

In [ ]:
import random
# 일단 min_size가 MIN_SIZE 보다 작게 리사이즈.
MIN_SIZE = 60
for anno_path, img_path in tqdm(pairs):
    with open(anno_path) as f:
        anno = json.load(f)
    pts = [np.array(e['points']) for e in anno['shapes']]
    boxes = [(e[:,0].min(), e[:,1].min(), e[:,0].max(), e[:,1].max()) for e in pts]
    min_size = min([min(x2-x1, y2-y1) for x1, y1, x2, y2 in boxes])
    ratio = 1
    im = cv2.imread(img_path)
    h, w = im.shape[:2]
    if MIN_SIZE < min_size and (128*128) < h*w:
        print(h,w)
        ratio = max(MIN_SIZE/min_size, np.sqrt((128*128)/(h*w)))  
        im = cv2.resize(im, (int(round(w*ratio)), int(round(h*ratio))), cv2.INTER_AREA)
        for obj in anno['shapes']:
            obj['points'] = (np.array(obj['points'])*ratio).tolist()
        cv2.imwrite(img_path, im, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
        h, w = im.shape[:2]
        #print(h,w)
        anno['imageHeight'] = h
        anno['imageWidth'] = w
        with open(anno_path, 'w') as f:
            json.dump(anno, f)
        #display(Image.open(img_path))


In [ ]:
merged = []
for anno_path, img_path in tqdm(pairs):
    with open(anno_path) as f:
        anno = json.load(f)
    w, h = anno['imageWidth'], anno['imageHeight']
    if 900 < w or 500 < h:
        print('not merged', w, h, img_path)
        #display(Image.open(img_path))
    else:
        merged.append((anno_path, img_path))
len(merged)

In [ ]:
for i in tqdm(range(0, len(merged), 4)):
    pairs = merged[i:i+4]
    if len(pairs) != 4:
        print('continue. len(pairs)==', len(pairs))
        continue
    def load(json_path, img_path):
        with open(json_path) as f:
            anno = json.load(f)
        im = cv2.imread(img_path)
        return anno, im
    datas = [load(*e) for e in pairs]
    hs = [im.shape[0] for _, im in datas]
    ws = [im.shape[1] for _, im in datas]
    max_h = max(hs)
    max_w = max(ws)
    
    max_h = max(hs[0] + hs[1], hs[0] + hs[3], hs[2] + hs[3], hs[2] + hs[1]) + 50 
    max_w = max(ws[0] + ws[2], ws[0] + ws[3], ws[1] + ws[3], ws[2] + ws[1]) + 50
    board = np.full((max_h, max_w, 3), 128, dtype=np.uint8)
    annos, ims = list(zip(*datas)) 
    
    board[:hs[0],  :ws[0]]  = ims[0]
    board[-hs[1]:, :ws[1]]  = ims[1]
    board[:hs[2],  -ws[2]:] = ims[2]
    board[-hs[3]:, -ws[3]:] = ims[3]

    for obj in annos[1]['shapes']:
        obj['points'] = (np.array(obj['points']) + (0, max_h-hs[1])).tolist()
    annos[0]['shapes'].extend(annos[1]['shapes'])
        
    for obj in annos[2]['shapes']:
        obj['points'] = (np.array(obj['points']) + (max_w-ws[2], 0)).tolist()
    annos[0]['shapes'].extend(annos[2]['shapes'])
    
    for obj in annos[3]['shapes']:
        obj['points'] = (np.array(obj['points']) + (max_w-ws[3], max_h-hs[3])).tolist()
    annos[0]['shapes'].extend(annos[3]['shapes'])
    
    annos[0]['imageHeight'] = max_h 
    annos[0]['imageWidth'] = max_w 
    
    for f in sum(pairs,()):
        os.remove(f)
    
    with open(pairs[0][0], 'w') as f:
        json.dump(annos[0], f)
    cv2.imwrite(pairs[0][1], board, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    

In [ ]:
pts = [np.array(obj['points']).astype(np.int)for obj in annos[0]['shapes']]
cv2.polylines(board, pts, True, (0, 255, 255))
Image.fromarray(board[:,:,[2,1,0]])